In [1]:
import scvi
import scanpy as sc

In [2]:
adata = scvi.data.pbmcs_10x_cite_seq(run_setup_anndata=False)

INFO     Downloading file at data/pbmc_10k_protein_v3.h5ad                                   
Downloading...: 24938it [00:00, 29001.53it/s]                             
INFO     Downloading file at data/pbmc_5k_protein_v3.h5ad                                    
Downloading...: 100%|██████████| 18295/18295.0 [00:00<00:00, 28708.67it/s]


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [17]:
import scvi

In [3]:
sc.pp.filter_genes(adata, min_counts=3)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [4]:
adata.layers["counts"] = adata.X.copy() # preserve counts 
sc.pp.normalize_total (adata, target_sum = 1e4)
sc.pp.log1p(adata)
adata.raw = adata 

In [5]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,
    subset=True,
    layer="counts",
    flavor="seurat_v3"
)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [6]:
scvi.data.setup_anndata(adata, layer="counts")

INFO     No batch_key inputted, assuming all cells are same batch                            
INFO     No label_key inputted, assuming all cells have same label                           
INFO     Using data from adata.layers["counts"]                                              
INFO     Computing library size prior per batch                                              
INFO     Successfully registered anndata object containing 10849 cells, 2000 vars, 1 batches,
         1 labels, and 0 proteins. Also registered 0 extra categorical covariates and 0 extra
         continuous covariates.                                                              
INFO     Please do not further modify adata until model is trained.                          


In [7]:
model = scvi.model.SCVI(adata)

/home/rsaha/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [8]:
model

In [9]:
model.train()

INFO     Training for 400 epochs                                                             
INFO     KL warmup for 400 epochs                                                            
Training...: 100%|██████████| 400/400 [19:32<00:00,  2.93s/it]
INFO     Training time:  1172 s. / 400 epochs                                                


In [10]:
model.save("my_model/")

In [11]:
latent = model.get_latent_representation()

In [12]:
adata.obsm["X_scVI"] = latent

In [15]:
adata.obs

,n_genes,percent_mito,n_counts,batch,_scvi_batch,_scvi_labels,_scvi_local_l_mean,_scvi_local_l_var
index,,,,,,,,
AAACCCAAGATTGTGA-1,2194,0.084903,6160.0,PBMC10k,0,0,6.824571,0.670148
AAACCCACATCGGTTA-1,2093,0.061820,6713.0,PBMC10k,0,0,6.824571,0.670148
AAACCCAGTACCGCGT-1,1518,0.078911,3637.0,PBMC10k,0,0,6.824571,0.670148
AAACCCAGTATCGAAA-1,737,0.088424,1244.0,PBMC10k,0,0,6.824571,0.670148
AAACCCAGTCGTCATA-1,1240,0.059747,2611.0,PBMC10k,0,0,6.824571,0.670148
...,...,...,...,...,...,...,...,...
TTTGGTTGTACGAGTG-1,1450,0.064818,5662.0,PBMC5k,0,0,6.824571,0.670148
TTTGTTGAGTTAACAG-1,3065,0.087742,10189.0,PBMC5k,0,0,6.824571,0.670148
TTTGTTGCAGCACAAG-1,1641,0.098523,4740.0,PBMC5k,0,0,6.824571,0.670148
